## Execute Task A Stunt

### Setup and Connect

In [45]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

class RobotControl():
    # Initialize Function
    def __init__(self, ble):
        self.ble = ble
        self.notifiers_on = False
        self.TOF_HISTORY_LEN = 60
        self.IMU_HISTORY_LEN = 300
        self.calibration_factor = 0

        # A variable to store the latest sensor value
        self.latest_tof_front_reading = None
        
        # A list to store the history of all the sensor values
        # Each item in the list is a tuple (value, time)
        # WARNING: The list could grow really fast; you need to deal with this accordingly.
        self.tof1_readings = []
        self.imu_readings = []
        self.bot_pid_debug = []
        
        # A variable to store the latest imu reading
        self.latest_imu_reading = None
        
        # Activate notifications (if required)
        self.setup_notify()
    
    # A function to activate various notifications (if required)
    def setup_notify(self):
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        ble.start_notify(ble.uuid['RX_IMU_PITCH'], self.imu_pitch_callback_handler)
        ble.start_notify(ble.uuid['RX_MOTOR_PID'], self.motor_pid_callback_handler)
        self.notifiers_on = True

    def stop_notify(self):
        ble.stop_notify(ble.uuid['RX_TOF1'])
        ble.stop_notify(ble.uuid['RX_IMU_PITCH'])
        ble.stop_notify(ble.uuid['RX_MOTOR_PID'])
        self.notifiers_on = False
    
    def tof1_callback_handler(self, uuid, byte_array):
        self.tof1_readings.append( self.ble.bytearray_to_float(byte_array) )
    
    def imu_pitch_callback_handler(self, uuid, byte_array):
        self.imu_readings.append( self.ble.bytearray_to_float(byte_array) )

    def motor_pid_callback_handler(self, uuid, byte_array):
        self.bot_pid_debug.append( self.ble.bytearray_to_float(byte_array) )

    def move_forward(self, speed):
        ble.send_command(CMD.MOVE_FORWARD, speed)
    
    def move_backward(self, speed):
        ble.send_command(CMD.MOVE_BACKWARD, speed)

    def tof1_callback_handler(self, uuid, byte_array):
        new_tof = self.ble.bytearray_to_float(byte_array)
        # print(f"Got TOF data: {new_tof}")
        self.tof1_readings.append(new_tof)

    def imu_pitch_callback_handler(self, uuid, byte_array):
        new_angle = self.ble.bytearray_to_float(byte_array)
        # print(f"Got angle data: {new_angle}")
        self.angle_data.append( new_angle )
        
    def set_params(self, pid_p_rot, pid_min_power_rot, pid_p_fwd, pid_min_power_fwd, imu_hz):
        self.ble.send_command(CMD.SET_PID, f"{pid_p_rot}|{pid_min_power_rot}|{pid_p_fwd}|{pid_min_power_fwd}|{imu_hz}"); 

    def set_motor_calib(self, calib_rot, calib_fwd):
        self.ble.send_command(CMD.SET_MOTOR_CALIB, f"{calib_rot}|{calib_fwd}")
        
    def move_distance(self, dist_mm):
        self.ble.send_command(CMD.MOVE_DISTANCE, dist_mm)

    def turn_degrees(self, angle):
        self.ble.send_command(CMD.TURN_DEGREES, angle)
    
    def stop(self):
        self.ble.send_command(CMD.STOP, None)

    def do_stunt(self, dist):
        self.ble.send_command(CMD.DO_STUNT, dist)
    
    def get_data(self, expected_len):
        # ble.send_command(CMD.GET_DATA, None)
        while len(self.tof1_readings) < expected_len or len(self.bot_pid_debug) < expected_len \
            or len(self.imu_readings) < expected_len:
            print(f"Awaiting data: {len(self.tof1_readings)} / {expected_len}")
            time.sleep(1)
            ble.send_command(CMD.GET_DATA, None)

    def spin(self, left, right, time_ms):
        l = left
        r = right
        self.ble.send_command(CMD.SPIN, f"{l}|{r}|{time_ms}")
        time.sleep(time_ms / 1000)

    def turn_degrees(self, angle):
        self.ble.send_command(CMD.TURN_DEGREES, angle)
        
    def move_duration(self, power, duration, pause=False):
        self.ble.send_command(CMD.MOVE_DURATION, f"{power}|{duration}")
        time.sleep(duration / 1000)
        if pause:
            robot.move_duration(0,250, pause=False)
            

    def dist_to_time(self, d):
        # distance (cm) to time (ms)
        # return round(69.04669* (d**0.644513))
        return round(60 * (d**0.644513))
    
    def move_dist_as_duration(self, dist, pause=False):
        t = self.dist_to_time(dist / 10)
        self.move_duration(75, t, pause=pause)
    


    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define Robot Controller Class

In [48]:
LOG.propagate = False
ble = get_ble_controller()
ble.connect()
time.sleep(3)

2022-05-14 15:39:24,397 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:07:21:8D:B3:44
2022-05-14 15:39:29,244 | INFO     |: Connected to C0:07:21:8D:B3:44


### Run Stunt

In [42]:
bot = RobotControl(ble)

In [74]:
bot.set_params(1.5, 80, 1.4, 35, 60)
bot.set_motor_calib(1.4, 1.3)

# Open-Loop Stunt (drift into spot)
# bot.move_duration(100,500)
# bot.spin(80,-80,900)
# bot.move_duration(100,1800)
# bot.spin(100,80,500)

# Parallel park
# bot.move_duration(0,500)
# bot.move_duration(100, 900)
# bot.stop()
# bot.turn_degrees(45)
# bot.stop()
# bot.move_duration(0,500)
# bot.spin(-80,-80,800)
# bot.stop()
# bot.move_duration(0,500)
# bot.turn_degrees(-40)
# bot.stop()

# Real stunt, like the flip thingy?

bot.spin(150,150,1000)
bot.spin(-200,-200,1000)
bot.stop()


# bot.move_distance(100)
# bot.do_stunt(900)
# bot.stop()

2022-05-14 15:55:42,705 | INFO     |: Disconnected from 7AE0A3CC-63D5-13D9-39BA-D29A7DC67D14


In [ ]:
bot.get_data(300)
print(f"TOF: {bot.tof1_readings}")
print(f"PID: {bot.bot_pid_debug}")
print(f"IMU (time): {bot.imu_readings}")

In [ ]:
print(f"TOF: {bot.tof1_readings}")
print(f"PID: {bot.bot_pid_debug}")
print(f"IMU (times): {bot.imu_readings}")
print(len(bot.bot_pid_debug))